In [3]:
import os

In [1]:
%pwd

'c:\\DANE\\Git Repos\\Kidney-Disease-Classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\DANE\\Git Repos\\Kidney-Disease-Classification'

In [12]:
# ============================
# Config dataclass
# ============================

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
  root_dir: Path
  source_url: str
  local_data_file: Path
  unzip_dir: Path

In [17]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories, get_size
from cnnClassifier import logger
import gdown
import zipfile 

In [15]:
# ============================
# Configuration Manager
# ============================

class ConfigurationManager:
    def __init__(
      self,
      config_filepath = CONFIG_FILE_PATH,
      params_filepath = PARAMS_FILE_PATH
      ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories(self.config.artifacts_root)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories(config.root_dir)

        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

In [18]:
# ============================
# Data Ingestion class
# ============================

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> Path:
        """Fetch data from Google Drive URL"""

        dataset_url = self.config.source_url
        zip_download_path = Path(self.config.local_data_file)

        os.makedirs(zip_download_path.parent, exist_ok=True)

        logger.info(
            f"Downloading data from {dataset_url} into file {zip_download_path}"
        )

        try:
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix + file_id, str(zip_download_path), quiet=False)
        except Exception as e:
            logger.error(f"Failed to download file from {dataset_url}: {e}")
            raise

        logger.info(f"Downloaded file size: {get_size(zip_download_path)}")
        return zip_download_path

    def extract_zip_file(self) -> None:
        """Extracts the downloaded ZIP file into the target directory"""

        unzip_path = Path(self.config.unzip_dir)
        os.makedirs(unzip_path, exist_ok=True)

        logger.info(f"Extracting {self.config.local_data_file} into {unzip_path}")

        try:
            with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
                zip_ref.extractall(unzip_path)
        except zipfile.BadZipFile as e:
            logger.error(f"Corrupted zip file {self.config.local_data_file}: {e}")
            raise
        except Exception as e:
            logger.error(
                f"Failed to extract zip file {self.config.local_data_file}: {e}"
            )
            raise

        logger.info(f"Extraction completed successfully: {unzip_path}")

In [ ]:
# ============================
# Pipeline execution
# ============================
if __name__ == "__main__":
    try:
        config_manager = ConfigurationManager()
        data_ingestion_config = config_manager.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)

        zip_file = data_ingestion.download_file()
        data_ingestion.extract_zip_file()
    except Exception as e:
        logger.exception(f"Data ingestion pipeline failed: {e}")
        raise

[2025-08-25 15:37:59,222 INFO - common : yaml file: config\config.yaml loaded successfully]
[2025-08-25 15:37:59,225 INFO - common : yaml file: params.yaml loaded successfully]
[2025-08-25 15:37:59,226 INFO - common : Created directory at: artifacts]
[2025-08-25 15:37:59,226 INFO - common : Created directory at: artifacts/data_ingestion]
